In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
import pandas as pd
import os

In [26]:
data = pd.read_csv('/content/drive/MyDrive/Final Fashion Dataset.csv')

image_dir = '/content/drive/MyDrive/images'

# Add a new column to the DataFrame with the full image path
data['Image_Path'] = data['filename'].apply(lambda x: os.path.join(image_dir, x))


In [27]:
data.head()

,user_id,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,ratings,review,Month,Price (USD),Image_Path
0,93810,8493,Women,Apparel,Bottomwear,Skirts,Black,Fall,2011.0,Casual,Forever New Women Black Skirts,8493.jpg,http://assets.myntassets.com/v1/images/style/p...,5,Amazing quality! Definitely recommend. Would r...,November,46,/content/drive/MyDrive/images/8493.jpg
1,24592,30757,Men,Apparel,Topwear,Kurtas,Grey,Summer,2012.0,Ethnic,Fabindia Men Grey Mangalgiri Kurta,30757.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy..",September,29,/content/drive/MyDrive/images/30757.jpg
2,13278,14881,Women,Accessories,Bags,Handbags,Black,Summer,2011.0,Casual,United Colors of Benetton Women Solid Black Ha...,14881.jpg,http://assets.myntassets.com/v1/images/style/p...,3,"It's okay, nothing special.!",October,39,/content/drive/MyDrive/images/14881.jpg
3,46048,48449,Men,Apparel,Topwear,Tshirts,Blue,Summer,2012.0,Casual,French Connection Men Blue T-shirt,48449.jpg,http://assets.myntassets.com/v1/images/style/p...,5,"Comfortable and stylish, worth the price. and ...",August,39,/content/drive/MyDrive/images/48449.jpg
4,42098,4697,Unisex,Accessories,Watches,Watches,Black,Winter,2016.0,Sports,ADIDAS Unisex Digital Duramo Black Watch,4697.jpg,http://assets.myntassets.com/v1/images/style/p...,4,"Perfect for any occasion, great buy.!",March,33,/content/drive/MyDrive/images/4697.jpg


In [32]:
# Display the unique class labels
unique_classes = data['subCategory'].unique()
print(f"Unique classes: {unique_classes}")
print(f"Number of unique classes: {len(unique_classes)}")


Unique classes: ['Bottomwear' 'Topwear' 'Bags' 'Watches' 'Shoes' 'Flip Flops' 'Jewellery'
 'Eyewear' 'Belts' 'Saree' 'Loungewear and Nightwear' 'Wallets' 'Socks']
Number of unique classes: 13


In [33]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

In [36]:
train_generator = train_datagen.flow_from_dataframe(
    data,
    x_col='Image_Path',
    y_col='subCategory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 29184 validated image filenames belonging to 13 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="Image_Path". These filename(s) will be ignored.
  warnings.warn(


In [38]:
validation_generator = train_datagen.flow_from_dataframe(
    data,
    x_col='Image_Path',
    y_col='subCategory',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 7295 validated image filenames belonging to 13 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="Image_Path". These filename(s) will be ignored.
  warnings.warn(


In [40]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to retain pretrained weights
base_model.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)      # Pooling layer to reduce spatial dimensions
x = Dense(512, activation='relu')(x) # Fully connected layer for feature learning
x = Dense(256, activation='relu')(x) # Another dense layer for additional feature extraction
predictions = Dense(len(unique_classes), activation='softmax')(x) # Output layer

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

In [41]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    verbose=1
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


406/912 ━━━━━━━━━━━━━━━━━━━━ 1:09:10 8s/step - accuracy: 0.4199 - loss: 1.9345